<a href="https://colab.research.google.com/github/KimberleyEvans-Parker/ALittleMidnightMusic/blob/master/keras_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/Shared drives/Part IV Project Resources/Recreation Notebooks/wav2mid-master

/content/gdrive/Shared drives/Part IV Project Resources/Recreation Notebooks/wav2mid-master


In [ ]:
import sys
# sys.path.append('.')

In [ ]:
pip install mido

     |████████████████████████████████| 61kB 1.7MB/s 


In [ ]:
pip install madmom

     |████████████████████████████████| 20.0MB 1.5MB/s 
  Created wheel for madmom: filename=madmom-0.16.1-cp36-cp36m-linux_x86_64.whl size=20940285 sha256=52f17e2efeeb09dfcaa95df86925c9d8c96a99362677b1892468320aa477ba96
  Stored in directory: /root/.cache/pip/wheels/21/0c/30/e0141aa75fb0a829ba5e1dca2be0860dc98502c1789616637d
Successfully built madmom


In [ ]:
pip install pretty_midi

     |████████████████████████████████| 5.6MB 2.4MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp36-none-any.whl size=5591953 sha256=d226bcd6afc85637a58d772d4ff6ba45705f4fa02ccf0fed80bd2b41197b1591
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


In [ ]:
import preprocess

In [ ]:
'''

keras: CNN Transcription model

'''
#from __future__ import print_function
import argparse
import matplotlib
import matplotlib.pyplot as plt

#keras utils
from keras.callbacks import Callback
from keras import metrics
from keras.models import Model, load_model
from keras.layers import Dense, Dropout, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, add
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
from keras.optimizers import SGD
from keras import backend as K
from keras.utils import plot_model


import tensorflow as tf
import sklearn
from sklearn.metrics import precision_recall_fscore_support

#internal utils
from preprocess import DataGen
from config import load_config

import numpy as np

import os


In [ ]:
def opt_thresholds(y_true,y_scores):
    othresholds = np.zeros(y_scores.shape[1])
    print(othresholds.shape)
    for label, (label_scores, true_bin) in enumerate(zip(y_scores.T,y_true.T)):
        #print label
        precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true_bin, label_scores)
        max_f1 = 0
        max_f1_threshold = .5
        for r, p, t in zip(recall, precision, thresholds):
            if p + r == 0: continue
            if (2*p*r)/(p + r) > max_f1:
                max_f1 = (2*p*r)/(p + r)
                max_f1_threshold = t
        #print label, ": ", max_f1_threshold, "=>", max_f1
        othresholds[label] = max_f1_threshold
        print(othresholds)
    return othresholds

In [ ]:
class linear_decay(Callback):
    '''
        decay = decay value to subtract each epoch
    '''
    def __init__(self, initial_lr,epochs):
        super(linear_decay, self).__init__()
        self.initial_lr = initial_lr
        self.decay = initial_lr/epochs

    def on_epoch_begin(self, epoch, logs={}):
        new_lr = self.initial_lr - self.decay*epoch
        print("ld: learning rate is now "+str(new_lr))
        K.set_value(self.model.optimizer.lr, new_lr)

In [ ]:
class half_decay(Callback):
    '''
        decay = decay value to subtract each epoch
    '''
    def __init__(self, initial_lr,period):
        super(half_decay, self).__init__()
        self.init_lr = initial_lr
        self.period = period

    def on_epoch_begin(self, epoch, logs={}):
        factor = epoch // self.period
        lr  = self.init_lr / (2**factor)
        print("hd: learning rate is now "+str(lr))
        K.set_value(self.model.optimizer.lr, lr)

In [ ]:
class Threshold(Callback):
    '''
        decay = decay value to subtract each epoch
    '''
    def __init__(self, val_data):
        super(Threshold, self).__init__()
        self.val_data = val_data
        _,y = val_data
        self.othresholds = np.full(y.shape[1],0.5)

    def on_epoch_end(self, epoch, logs={}):
        #find optimal thresholds on validation data
        x,y_true = self.val_data
        y_scores = self.model.predict(x)
        self.othresholds = opt_thresholds(y_true,y_scores)
        y_pred = y_scores > self.othresholds
        p,r,f,s = sklearn.metrics.precision_recall_fscore_support(y_true,y_pred,average='micro')
        print("validation p,r,f,s:")
        print (p,r,f,s)

In [ ]:
def baseline_model():
    inputs = Input(shape=input_shape)
    reshape = Reshape(input_shape_channels)(inputs)

    #normal convnet layer (have to do one initially to get 64 channels)
    conv1 = Conv2D(50,(5,25),activation='tanh')(reshape)
    do1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling2D(pool_size=(1,3))(do1)

    conv2 = Conv2D(50,(3,5),activation='tanh')(pool1)
    do2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling2D(pool_size=(1,3))(do2)

    flattened = Flatten()(pool2)
    fc1 = Dense(1000, activation='sigmoid')(flattened)
    do3 = Dropout(0.5)(fc1)

    fc2 = Dense(200, activation='sigmoid')(do3)
    do4 = Dropout(0.5)(fc2)
    outputs = Dense(note_range, activation='sigmoid')(do4)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def resnet_model(bin_multiple):

    #input and reshape
    inputs = Input(shape=input_shape)
    reshape = Reshape(input_shape_channels)(inputs)

    #normal convnet layer (have to do one initially to get 64 channels)
    conv = Conv2D(64,(1,bin_multiple*note_range),padding="same",activation='relu')(reshape)
    pool = MaxPooling2D(pool_size=(1,2))(conv)

    for i in range(int(np.log2(bin_multiple))-1):
        print(i)
        #residual block
        bn = BatchNormalization()(pool)
        re = Activation('relu')(bn)
        freq_range = (bin_multiple/(2**(i+1)))*note_range
        print(freq_range)
        conv = Conv2D(64,(1,freq_range),padding="same",activation='relu')(re)

        #add and downsample
        ad = add([pool,conv])
        pool = MaxPooling2D(pool_size=(1,2))(ad)

    flattened = Flatten()(pool)
    fc = Dense(1024, activation='relu')(flattened)
    do = Dropout(0.5)(fc)
    fc = Dense(512, activation='relu')(do)
    do = Dropout(0.5)(fc)
    outputs = Dense(note_range, activation='sigmoid')(do)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
window_size = 7
min_midi = 21
max_midi = 108
note_range = max_midi - min_midi + 1


def train(args):
    print("training...")
    path = os.path.join('models',args['model_name'])
    config = load_config(os.path.join(path,'config.json'))

    global feature_bins
    global input_shape
    global input_shape_channels

    bin_multiple = int(args['bin_multiple'])
    print('bin multiple',str(np.log2(bin_multiple)))
    feature_bins = note_range * bin_multiple
    input_shape = (window_size,feature_bins)
    input_shape_channels = (window_size,feature_bins,1)

    #filenames
    model_ckpt = os.path.join(path,'ckpt.h5')

    #train params
    batch_size = 256
    epochs = 1000

    trainGen = DataGen(os.path.join(path,'data','train'),batch_size,args)
    valGen = DataGen(os.path.join(path,'data','val'),batch_size,args)
    #valData = load_data(os.path.join(path,'data','val'))


    if os.path.isfile(model_ckpt):
        print('loading model')
        model = load_model(model_ckpt)
    else:
        print('training new model from scratch')
        if bool(args['residual']):
            model = resnet_model(bin_multiple)
        else:
            model = baseline_model()

    init_lr = float(args['init_lr'])

    model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=init_lr,momentum=0.9))
    model.summary()
    plot_model(model, to_file=os.path.join(path,'model.png'))

    checkpoint = ModelCheckpoint(model_ckpt, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = EarlyStopping(patience=5,monitor='val_loss', verbose=1, mode='min')
    #tensorboard = TensorBoard(log_dir='./logs/baseline/', histogram_freq=250, batch_size=batch_size)
    if args['lr_decay'] == 'linear':
        decay = linear_decay(init_lr,epochs)
    else:
        decay = half_decay(init_lr,5)
    csv_logger = CSVLogger(os.path.join(path,'training.log'))
    #t = Threshold(valData)
    callbacks = [checkpoint,early_stop,decay,csv_logger]

    history = model.fit_generator(trainGen.next(),trainGen.steps(), epochs=epochs,
              verbose=1,validation_data=valGen.next(),validation_steps=valGen.steps(),callbacks=callbacks)

    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    '''plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('baseline/acc.png')'''

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('baseline/loss.png')

    #test
    testGen = DataGen(os.path.join(path,'data','test'),batch_size,args)

    res = model.evaluate_generator(testGen.next(),steps=testGen.steps())
    print(model.metrics_names)
    print(res)

In [ ]:
def main():
    print("maining...")
    #train
    parser = argparse.ArgumentParser(
        description='Preprocess MIDI/Audio file pairs into ingestible data')
    parser.add_argument('model_name',
                        help='Path to the model directory where data should reside')
    # parser.add_argument('-f')

    args = vars(parser.parse_args())
    train(args)


if __name__ == '__main__':
    main()

maining...


usage: ipykernel_launcher.py [-h] model_name
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def opt_thresholds(y_true,y_scores):
    othresholds = np.zeros(y_scores.shape[1])
    print(othresholds.shape)
    for label, (label_scores, true_bin) in enumerate(zip(y_scores.T,y_true.T)):
        #print label
        precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true_bin, label_scores)
        max_f1 = 0
        max_f1_threshold = .5
        for r, p, t in zip(recall, precision, thresholds):
            if p + r == 0: continue
            if (2*p*r)/(p + r) > max_f1:
                max_f1 = (2*p*r)/(p + r)
                max_f1_threshold = t
        #print label, ": ", max_f1_threshold, "=>", max_f1
        othresholds[label] = max_f1_threshold
        print(othresholds)
    return othresholds

class linear_decay(Callback):
    '''
        decay = decay value to subtract each epoch
    '''
    def __init__(self, initial_lr,epochs):
        super(linear_decay, self).__init__()
        self.initial_lr = initial_lr
        self.decay = initial_lr/epochs

    def on_epoch_begin(self, epoch, logs={}):
        new_lr = self.initial_lr - self.decay*epoch
        print("ld: learning rate is now "+str(new_lr))
        K.set_value(self.model.optimizer.lr, new_lr)

class half_decay(Callback):
    '''
        decay = decay value to subtract each epoch
    '''
    def __init__(self, initial_lr,period):
        super(half_decay, self).__init__()
        self.init_lr = initial_lr
        self.period = period

    def on_epoch_begin(self, epoch, logs={}):
        factor = epoch // self.period
        lr  = self.init_lr / (2**factor)
        print("hd: learning rate is now "+str(lr))
        K.set_value(self.model.optimizer.lr, lr)

class Threshold(Callback):
    '''
        decay = decay value to subtract each epoch
    '''
    def __init__(self, val_data):
        super(Threshold, self).__init__()
        self.val_data = val_data
        _,y = val_data
        self.othresholds = np.full(y.shape[1],0.5)

    def on_epoch_end(self, epoch, logs={}):
        #find optimal thresholds on validation data
        x,y_true = self.val_data
        y_scores = self.model.predict(x)
        self.othresholds = opt_thresholds(y_true,y_scores)
        y_pred = y_scores > self.othresholds
        p,r,f,s = sklearn.metrics.precision_recall_fscore_support(y_true,y_pred,average='micro')
        print("validation p,r,f,s:")
        print (p,r,f,s)

def baseline_model():
    inputs = Input(shape=input_shape)
    reshape = Reshape(input_shape_channels)(inputs)

    #normal convnet layer (have to do one initially to get 64 channels)
    conv1 = Conv2D(50,(5,25),activation='tanh')(reshape)
    do1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling2D(pool_size=(1,3))(do1)

    conv2 = Conv2D(50,(3,5),activation='tanh')(pool1)
    do2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling2D(pool_size=(1,3))(do2)

    flattened = Flatten()(pool2)
    fc1 = Dense(1000, activation='sigmoid')(flattened)
    do3 = Dropout(0.5)(fc1)

    fc2 = Dense(200, activation='sigmoid')(do3)
    do4 = Dropout(0.5)(fc2)
    outputs = Dense(note_range, activation='sigmoid')(do4)

    model = Model(inputs=inputs, outputs=outputs)
    return model



def resnet_model(bin_multiple):

    #input and reshape
    inputs = Input(shape=input_shape)
    reshape = Reshape(input_shape_channels)(inputs)

    #normal convnet layer (have to do one initially to get 64 channels)
    conv = Conv2D(64,(1,bin_multiple*note_range),padding="same",activation='relu')(reshape)
    pool = MaxPooling2D(pool_size=(1,2))(conv)

    for i in range(int(np.log2(bin_multiple))-1):
        print(i)
        #residual block
        bn = BatchNormalization()(pool)
        re = Activation('relu')(bn)
        freq_range = (bin_multiple/(2**(i+1)))*note_range
        print(freq_range)
        conv = Conv2D(64,(1,freq_range),padding="same",activation='relu')(re)

        #add and downsample
        ad = add([pool,conv])
        pool = MaxPooling2D(pool_size=(1,2))(ad)

    flattened = Flatten()(pool)
    fc = Dense(1024, activation='relu')(flattened)
    do = Dropout(0.5)(fc)
    fc = Dense(512, activation='relu')(do)
    do = Dropout(0.5)(fc)
    outputs = Dense(note_range, activation='sigmoid')(do)

    model = Model(inputs=inputs, outputs=outputs)

    return model

window_size = 7
min_midi = 21
max_midi = 108
note_range = max_midi - min_midi + 1


def train(args):
    print("training...")
    path = os.path.join('models',args['model_name'])
    config = load_config(os.path.join(path,'model.json'))

    global feature_bins
    global input_shape
    global input_shape_channels

    bin_multiple = int(args['bin_multiple'])
    print('bin multiple',str(np.log2(bin_multiple)))
    feature_bins = note_range * bin_multiple
    input_shape = (window_size,feature_bins)
    input_shape_channels = (window_size,feature_bins,1)

    #filenames
    model_ckpt = os.path.join(path,'ckpt.h5')

    #train params
    batch_size = 256
    epochs = 1000

    trainGen = DataGen(os.path.join(path,'data','train'),batch_size,args)
    valGen = DataGen(os.path.join(path,'data','val'),batch_size,args)
    #valData = load_data(os.path.join(path,'data','val'))


    if os.path.isfile(model_ckpt):
        print('loading model')
        model = load_model(model_ckpt)
    else:
        print('training new model from scratch')
        if bool(args['residual']):
            model = resnet_model(bin_multiple)
        else:
            model = baseline_model()

    init_lr = float(args['init_lr'])

    model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=init_lr,momentum=0.9))
    model.summary()
    plot_model(model, to_file=os.path.join(path,'model.png'))

    checkpoint = ModelCheckpoint(model_ckpt, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = EarlyStopping(patience=5,monitor='val_loss', verbose=1, mode='min')
    #tensorboard = TensorBoard(log_dir='./logs/baseline/', histogram_freq=250, batch_size=batch_size)
    if args['lr_decay'] == 'linear':
        decay = linear_decay(init_lr,epochs)
    else:
        decay = half_decay(init_lr,5)
    csv_logger = CSVLogger(os.path.join(path,'training.log'))
    #t = Threshold(valData)
    callbacks = [checkpoint,early_stop,decay,csv_logger]

    history = model.fit_generator(trainGen.next(),trainGen.steps(), epochs=epochs,
              verbose=1,validation_data=valGen.next(),validation_steps=valGen.steps(),callbacks=callbacks)

    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    '''plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('baseline/acc.png')'''

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('baseline/loss.png')

    #test
    testGen = DataGen(os.path.join(path,'data','test'),batch_size,args)

    res = model.evaluate_generator(testGen.next(),steps=testGen.steps())
    print(model.metrics_names)
    print(res)
def main():
    print("maining...")
    #train
    # parser = argparse.ArgumentParser(
    #     description='Preprocess MIDI/Audio file pairs into ingestible data')
    # parser.add_argument('model_name',
    #                     help='Path to the model directory where data should reside')
    # parser.add_argument('-f')

    # args = vars(parser.parse_args())
    # train(args)
    train({"model_name": "new", "bin_multiple": 4, "init_ltr": 0.1, "lr_decay": "half", "residual": True, "spec_type": "cqt"})


if __name__ == '__main__':
    main()

maining...
training...
bin multiple 2.0
initializing gen for models/new/data/train
Bounce


ValueError: ignored

In [ ]:
%tb


IndexError: ignored

In [ ]:
!python keras_train.py new


Using TensorFlow backend.
2020-07-28 22:48:37.459279: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "keras_train.py", line 267, in <module>
    main()
  File "keras_train.py", line 263, in main
    train(args)
  File "keras_train.py", line 173, in train
    config = load_config(os.path.join(path,'config.py'))
  File "/content/gdrive/Shared drives/Part IV Project Resources/Recreation Notebooks/wav2mid-master/config.py", line 12, in load_config
    with open(json_fn, 'r') as infile:
FileNotFoundError: [Errno 2] No such file or directory: 'models/new/config.py'
